In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import os.path as op
import numpy as np
import seaborn as sns
import pyro

import sys
sys.path.insert(0, '../')

from spectrum.preprocessing import encoders
from spectrum.judge import lca, utils

from spectrum import evaluator

Create some data

In [ ]:
claims = dict()
claims['source_id'] = [0, 0, 1, 1]
claims['object_id'] = [0, 1, 1, 0]
claims['value'] = [0, 1, 0, 1]
# claims['domain_size'] = [2, 2, 2, 2]
claims = pd.DataFrame(data=claims)

mask = lca.build_mask(claims)
observation = lca.build_observation(claims)

### Model with observation

In [ ]:
claims

In [ ]:
utils.print_trace(pyro.poutine.trace(lca.lca_model).get_trace(claims))

In [ ]:
data = lca.make_observation_mapper(observation, mask)
conditioned_lca = pyro.condition(lca.lca_model, data=data)

for i in range(3):
    utils.print_trace(pyro.poutine.trace(conditioned_lca).get_trace(observation, mask))
    print('_' * 10)

### Guide

In [ ]:
for i in range(3):
    utils.print_trace(pyro.poutine.trace(lca.lca_guide).get_trace(observation, mask))
    print('_' * 10)

In [ ]:
params = pyro.get_param_store()

In [ ]:
params['theta_s_0']

In [ ]:
def generate_one_simpleLCA_sample(observation, mask):
    tracer = pyro.poutine.trace(conditioned_lca)
    trace = tracer.get_trace(observation, mask)

    for name, node in trace.nodes.items():
        if node['type'] == 'sample':
            print(f'{node["name"]} - sampled value {node["value"]} ')
    return trace